In [ ]:
%load_ext autoreload
%autoreload 2
import time
from splinter import Browser
from stem import Signal
from stem.control import Controller
from IPython.display import clear_output
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import matplotlib.pyplot as plt
import json
import datetime
import tinydb
import io
import regex
import youtube
import common
#%load_ext snakeviz

In [ ]:
s = common.Settings()
print([s.get_parameter('address'),
s.get_parameter('login'),
s.get_parameter('password')])

In [ ]:
generator = common.CommentGenerator()
text = generator.get_comment()
print common.CommentGenerator.index
print text.encode('utf8')

In [ ]:
#test
y = youtube.Youtube()
y.login()
#y.get_user_subscribers('https://www.youtube.com/channel/UCHR-wOd9AxHq89d61QnFdiQ')
y.browser.visit('https://www.youtube.com/watch?v=i3IzFITKaVw')
print y.browser.title, y.have_it_cyrillic_letters(y.browser.title)
comment = y.comment_on_video()
time.sleep(20)
if comment:
    y.comments.add(user=user, video=video, comment = comment)
#y.browser.quit()

In [ ]:
y.browser.quit()

In [ ]:
#программа комментирования видео и подписки
y = youtube.Youtube()  
try:
    y.login()
except:
    y.browser.quit()
    raise Exception('login error')    
#subs_cache = []
if not 'subs_cache' in globals() or len(subs_cache) == 0: 
    y.get_subscriptions()
    subs_cache = y.subscriptions
else:
    # берем подписчиков из кэша
    for subs in subs_cache:
        y.subscriptions.append(common.Subscription(subs.url, subs.name))
print 'Наших подписок: {}'.format(len(y.subscriptions))
if len(common.Subscription.url_list) == 0:
    raise Exception('url list is empty!')
i = 0
#video_commented = False
comment = ''
max_from_user = 10 # сколько подписчиков берем с 1-го пользователя
max_comments_limit = 200
channels = y.get_channel_list()
#for subs in reversed(channels): # каналы начиная с самой первой подписки по времени
for subs in channels: # цикл по нашим подпискам
    #u_subs = y.get_user_subscribers(subs.url)
    take_from_user = 0
    u_subs = y.get_user_subscribers(subs)
    for user in u_subs: # цикл по подпискам пользователя
        take_from_user += 1
        if take_from_user > max_from_user: 
            print 'Подписчик {} от текущего пользователя, переходим к следующему'.format(max_from_user)
            continue        
        if not user in common.Subscription.url_list:
            if not y.comments.is_user_commented(user):
                amount_subs = y.open_user_page(user)
                if amount_subs < y.max_subscribers_amount:
                    video = y.open_last_user_video(user)
                    comment = y.comment_on_video()
                    if comment:
                        y.comments.add(user=user, video=video, comment = comment)
                    i += 1
                    if i == max_comments_limit:
                        y.browser.quit()
                        raise Exception('Достигнут предел {} комментариев'.format(max_comments_limit))

if not comment:
    #print y.log.info('Не откомментировали последнее видео, возможно нужно обновить список каналов с которых берется подписка!')
    result = y.browser.quit()
subs_cache = y.subscriptions

In [ ]:
commentators = []
def print_comments():
    c_list = y.browser.find_by_xpath('//*[@class="comment-entry"]')
    print len(c_list)
    for comment in c_list:
        #print comment
        text = comment.find_by_tag('div')
        for div in text:
            if div['class'] == 'comment-text-content':
                print div.value
        links = comment.find_by_tag('a')
        user = links.first['href']
        if not user in commentators:
            commentators.append(user)
        
y = Youtube()
y.login()
y.browser.visit('https://www.youtube.com/comments')
while 1==1:
    page = y.browser.find_by_xpath('//*[@id="yt-comments-paginator"]')
    #print page
    #print page.first
    if len(page) == 0 or not page.first.visible:
        break
    else:
        page.first.click()
print_comments()
y.browser.visit('https://www.youtube.com/messages')
print_comments()

In [ ]:
y.browser.quit()

In [ ]:
#Список откоментированных видео
#y = Youtube()
#y.comments.is_video_commented('dsfs')
#print user, y.comments.is_user_commented(user)
y.comments.indent()
c_list = y.comments.db.all()
c_list = sorted(c_list, key = lambda item:item['date'])
print 'Всего количество комментариев {}'.format(len(c_list))
for c in y.comments.db.all():
    print datetime.datetime.fromtimestamp(c['date']), c['video'], c['comment'] #c['user'],
#y.press_subscribe()
#y.open_last_user_video(user)